<a href="https://colab.research.google.com/github/ByungjunKim/JKFR-100th-Question/blob/main/Author_id(KAC)_%EB%A7%A4%EC%B9%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install thefuzz python-Levenshtein

In [ ]:
import pandas as pd
import json
from rapidfuzz import process, fuzz, utils
from tqdm import tqdm

# 1. 파일 경로 설정
csv_file = '/content/[현대소설연구 번역서, 원서 참고문헌 최종] (논문 발행연도 포함).csv'
json_file = '/content/Person_slim_all (1).json'

# 2. 데이터 로드
df = pd.read_csv(csv_file)
with open(json_file, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

df['제어번호'] = None
df['매칭점수'] = 0.0 # Initialize as float to prevent FutureWarning

# 3. 헬퍼 함수: 다양한 타입을 문자열로 통일
def extract_name_string(name_data):
    if isinstance(name_data, str): return name_data.strip()
    if isinstance(name_data, dict): return name_data.get('label') or name_data.get('value') or str(name_data)
    if isinstance(name_data, list): return extract_name_string(name_data[0]) if name_data else ""
    return str(name_data) if name_data else ""

# 4. JSON 데이터 색인 (label + altlabel 포함)
print("LOD DB 색인 중 (약 100만 건)...")
name_to_id_map = {}
for item in json_data:
    target_id = item.get('id')
    candidates = [extract_name_string(item.get('label'))]
    alts = item.get('altlabel') or item.get('alterlabel') or []
    if isinstance(alts, list):
        candidates.extend([extract_name_string(a) for a in alts])
    else:
        candidates.append(extract_name_string(alts))

    for name in set(filter(None, candidates)):
        name_to_id_map[name] = target_id

lookup_names = list(name_to_id_map.keys())

# 5. 매칭 함수 (RapidFuzz 기반)
def perform_rapid_match(name):
    if pd.isna(name) or str(name).strip() == '':
        return None, 0

    # 100만 건 대조를 위해 score_cutoff 설정 (연산 절약)
    result = process.extractOne(
        str(name),
        lookup_names,
        scorer=fuzz.token_sort_ratio,
        processor=utils.default_process,
        score_cutoff=60  # 최소 60점 이상만 후보로 고려
    )

    if result:
        matched_name, score, _ = result
        return name_to_id_map.get(matched_name), score
    return None, 0

# 6. 실행 (캐싱 활용으로 유니크 작가만 계산)
print("🚀 매칭 시작 (100만 건 대상)...")
unique_authors = df['author_정제'].dropna().unique()
match_cache = {}

for name in tqdm(unique_authors, desc="작가명 매칭 중"):
    match_cache[name] = perform_rapid_match(name)

# 7. 결과 반영 및 필터링 (85점 기준)
df['match_res'] = df['author_정제'].map(match_cache)
df['temp_id'] = df['match_res'].apply(lambda x: x[0] if isinstance(x, tuple) else None)
df['temp_score'] = df['match_res'].apply(lambda x: x[1] if isinstance(x, tuple) else 0.0) # Ensure 0 is float

# [조건 적용] 85점 이상만 제어번호와 점수를 확정 기록
confirm_mask = df['temp_score'] >= 85
df.loc[confirm_mask, '제어번호'] = df.loc[confirm_mask, 'temp_id']
df.loc[confirm_mask, '매칭점수'] = df.loc[confirm_mask, 'temp_score']

# 8. 미검수 리스트 추출 (85점 미만 또는 매칭 실패)
# 제어번호가 비어있는 모든 데이터를 추출합니다.
unverified_df = df[df['제어번호'].isna()][['author_정제', 'title_정제', 'temp_id', 'temp_score']].copy()
unverified_df.columns = ['작가명_정제', '작품명_정제', '추천_id', '유사도점수']

# 9. 파일 저장
# 최종 결과 파일 (임시 컬럼 제외)
df.drop(columns=['match_res', 'temp_id', 'temp_score']).to_csv('result_step1.csv', index=False, encoding='utf-8-sig')
# 미검수 리스트 저장
unverified_df.to_csv('미검수_작가_리스트.csv', index=False, encoding='utf-8-sig')

print("-" * 50)
print(f"✅ 작업 완료!")
print(f" - 자동 매칭 성공 (85점 이상): {confirm_mask.sum():,} 건")
print(f" - 미검수 리스트 생성: {len(unverified_df):,} 건")
print("-" * 50)